In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/assessmentanalysis/

/content/drive/MyDrive/assessmentanalysis


In [ ]:
#Installing the required packages for this project
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate
!pip install ffmpeg-python
!pip install firebase-admin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 112.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Data Extration

In [4]:
from transformers import AutoProcessor, AutoModelForCausalLM
import os
import ffmpeg

In [5]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from firebase_admin import firestore

cred = credentials.Certificate('videoanalysis-d5eb4-firebase-adminsdk-u5vqo-754ba234c2.json')
firebase_admin.initialize_app(cred, {
    'storageBucket': 'videoanalysis-d5eb4.firebasestorage.app'
})

bucket = storage.bucket()
db = firestore.client()

In [6]:
file_name="CodingChallenge.mp4"

In [ ]:
if os.path.exists(file_name):
  print(f"The file '{file_name}' exists.")
else:
  video_path_in_firebase = "videos/"+file_name  # Path
  local_video_path = file_name  # Local path for the downloaded file
  blob = bucket.blob(video_path_in_firebase)  # Get the blob
  blob.download_to_filename(local_video_path)
  print(f"Video downloaded to: {local_video_path}")

The file 'CodingChallenge.mp4' exists.


In [8]:
video_name = os.path.basename(file_name)
video_name_without_extension = os.path.splitext(video_name)[0]

In [9]:
!pip install openai-whisper

import whisper
model = whisper.load_model('large')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 21.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=9a165bf234c57197f79cff84ddd8ecf00f1e5c2140526cc7a6dfc0dc93835710
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


100%|█████████████████████████████████████| 2.88G/2.88G [00:48<00:00, 64.3MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
def convert_mp4_to_wav(input_file, output_file):
  """Converts an MP4 file to a WAV file.

  Args:
    input_file: Path to the input MP4 file.
    output_file: Path to the output WAV file.
  """
  try:
    (
        ffmpeg
        .input(input_file)
        .output(output_file, acodec='pcm_s16le', ac=1, ar='16k')
        .run(overwrite_output=True)
    )
    print(f"Conversion successful: {input_file} -> {output_file}")
  except ffmpeg.Error as e:
    print(f"Error during conversion: {e}")

# Check and insert in the 'text' collection in Firebase
text_doc_ref = db.collection('text').document(video_name_without_extension)
text_doc = text_doc_ref.get()
audiofile=video_name_without_extension+".wav"

if not text_doc.exists:
  # Using convert_mp4_to_wav
  audio_file_exist=os.path.exists(audiofile)
  if not audio_file_exist:
    input_mp4_file = file_name
    convert_mp4_to_wav(input_mp4_file, audiofile)

  # Using whisper
  data=model.transcribe(audio=audiofile, language='en', verbose=True)
  audio_chunks=data["segments"]

  text_data = {
      "filename": video_name_without_extension,
      "content": audio_chunks
  }

  text_doc_ref.set(text_data)
  print(f"Document added to 'text' collection for filename: {video_name_without_extension}")
else:
  print(f"Document already exists in 'text' collection for filename: {video_name_without_extension}")

Conversion successful: CodingChallenge.mp4 -> CodingChallenge.wav
[00:00.000 --> 00:06.500]  Okay then my friends, so in this challenge I'm going to ask you to make a simple toggle switch
[00:06.500 --> 00:12.300]  just like this using a checkbox input field and some CSS. So no JavaScript whatsoever,
[00:12.580 --> 00:17.760]  just HTML and CSS. Now to begin with, that's going to look something like this in a browser,
[00:17.760 --> 00:23.400]  just an ordinary checkbox. So what I want you to do is take this checkbox and make a toggle switch
[00:23.400 --> 00:29.800]  like this based on that checkbox. Now one thing to note, it's not the checkbox itself,
[00:30.000 --> 00:34.720]  that's being styled to make it look like this. The checkbox is just there under the hood for
[00:34.720 --> 00:39.980]  the functionality of being checked as the actual form field when we toggle the switch on and off.
[00:40.380 --> 00:45.220]  So when we toggle it on, then the checkbox is essentially ticked. 